In [1]:
from spimi import SPIMI
import pandas as pd

[nltk_data] Downloading package punkt to /home/ambar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# testeo
path = './dataset/'
data_path = path + 'spotify_songs_test2.csv'

data = pd.read_csv(data_path)

In [4]:
spimi = SPIMI(size_per_block=10240*4,
              path_block= './.temp1/',
              output_folder='./blocks1/',
              ram_limit=1024*1024*1024*4,
              size_per_block_out= 1024*4)

spimi.BSBIndexConstuction(data)

['./blocks1/block_0.txt',
 './blocks1/block_1.txt',
 './blocks1/block_2.txt',
 './blocks1/block_3.txt',
 './blocks1/block_4.txt',
 './blocks1/block_5.txt',
 './blocks1/block_6.txt',
 './blocks1/block_7.txt',
 './blocks1/block_8.txt',
 './blocks1/block_9.txt',
 './blocks1/block_10.txt',
 './blocks1/block_11.txt',
 './blocks1/block_12.txt',
 './blocks1/block_13.txt',
 './blocks1/block_14.txt',
 './blocks1/block_15.txt',
 './blocks1/block_16.txt',
 './blocks1/block_17.txt',
 './blocks1/block_18.txt',
 './blocks1/block_19.txt',
 './blocks1/block_20.txt',
 './blocks1/block_21.txt',
 './blocks1/block_22.txt',
 './blocks1/block_23.txt',
 './blocks1/block_24.txt',
 './blocks1/block_25.txt',
 './blocks1/block_26.txt',
 './blocks1/block_27.txt',
 './blocks1/block_28.txt',
 './blocks1/block_29.txt',
 './blocks1/block_30.txt',
 './blocks1/block_31.txt',
 './blocks1/block_32.txt',
 './blocks1/block_33.txt',
 './blocks1/block_34.txt',
 './blocks1/block_35.txt',
 './blocks1/block_36.txt',
 './blocks1

In [46]:
import numpy as np
import os
import re
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import pandas as pd
import bisect

class CosineSimilaritySearch:
    def __init__(self, block_folder):
        self.block_folder = block_folder
        self.language_map = {
            'es': 'spanish',
            'en': 'english',
            'fr': 'french',
            'de': 'german',
            'it': 'italian'
        }

    def preprocess(self, text, lang='en'):
        """Preprocesa el texto de la consulta según el idioma especificado, manteniendo frases importantes."""
        lang = self.language_map.get(lang, 'english')
        stop_words = stopwords.words(lang)
        stemmer = SnowballStemmer(language=lang)
        
        # Procesa el texto para conservar frases exactas
        text = text.lower()
        text = re.sub(r'[^a-zA-Z0-9_À-ÿ]', ' ', text)
        
        # Detectar frases exactas que deben mantenerse sin cambios
        important_phrases = ["mayor que yo"]  # Aquí puedes añadir más frases que deben conservarse tal cual
        for phrase in important_phrases:
            if phrase in text:
                text = text.replace(phrase, "_".join(phrase.split()))  # Reemplaza espacios con guiones bajos para preservar la frase

        words = text.split()
        processed_words = []
        
        for word in words:
            if "_" in word:  # Si es una frase marcada, no la dividimos ni la procesamos
                processed_words.append(word)
            elif word not in stop_words:
                processed_words.append(stemmer.stem(word))
        
        # Restaurar los espacios en frases que se mantuvieron
        return [w.replace("_", " ") if "_" in w else w for w in processed_words]
    def calculate_query_vector(self, query_terms, df_dict, num_docs):
        tf_query = defaultdict(int)
        for term in query_terms:
            tf_query[term] += 1
        
        query_vector = {}
        for term, tf in tf_query.items():
            idf = np.log((num_docs / df_dict[term])) if term in df_dict else 0
            query_vector[term] = tf * idf
        return query_vector

    def load_block_terms(self, query_terms):
        term_postings = {}
        df_dict = {}
        num_docs = 0

        for filename in os.listdir(self.block_folder):
            with open(os.path.join(self.block_folder, filename), 'r') as file:
                for line in file:
                    term, rest = line.split(" (DF: ")
                    if self.binary_search(query_terms, term):
                        df, postings = rest.split("): ")
                        df = int(df)
                        df_dict[term] = df
                        num_docs += df
                        
                        term_postings[term] = []
                        postings_list = postings.strip().split("), (")
                        for posting in postings_list:
                            doc_id, tf = map(int, posting.strip("()").split(", "))
                            term_postings[term].append((doc_id, tf))
        return term_postings, df_dict, num_docs

    def binary_search(self, sorted_list, item):
        index = bisect.bisect_left(sorted_list, item)
        return index < len(sorted_list) and sorted_list[index] == item

    def cosine_similarity(self, query_vector, doc_vectors):
        doc_scores = defaultdict(float)
        query_norm = np.sqrt(np.sum(np.square(list(query_vector.values()))))

        for term, query_weight in query_vector.items():
            if term in doc_vectors:
                for doc_id, doc_tf in doc_vectors[term]:
                    doc_weight = doc_tf * query_weight
                    doc_scores[doc_id] += doc_weight

        # Normalización y cálculo de similitud de coseno
        for doc_id in doc_scores:
            doc_norm = np.sqrt(np.sum([
                (doc_tf * query_vector.get(term, 0))**2
                for term in query_vector
                for doc_id_in_posting, doc_tf in doc_vectors.get(term, [])
                if doc_id_in_posting == doc_id
            ]))
            if doc_norm != 0 and query_norm != 0:
                doc_scores[doc_id] /= (query_norm * doc_norm)
            else:
                doc_scores[doc_id] = 0  # Evitar división por cero

        return sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)

    def get_top_k_similar_documents(self, query, data, lang='en', k=5):
        query_terms = self.preprocess(query, lang=lang)
        print(query_terms)
        term_postings, df_dict, num_docs = self.load_block_terms(query_terms)
        query_vector = self.calculate_query_vector(query_terms, df_dict, num_docs)
        
        doc_scores = self.cosine_similarity(query_vector, term_postings)
        doc_scores = sorted(doc_scores, key=lambda x: x[1], reverse=True)[:k]

        results = []
        for doc_id, score in doc_scores:
            try:
                doc_details = data.iloc[doc_id].to_dict()
                doc_details["Cosine Similarity Score"] = score
                results.append(doc_details)
            except IndexError:
                print(f"El documento con ID {doc_id} no está disponible en los datos.")

        results_df = pd.DataFrame(results)
        results_df = results_df[["track_name", "track_artist", "lyrics", "Cosine Similarity Score"]]
        results_df.columns = ["Song Title", "Artist", "Lyrics", "Similarity Score"]
        results_df.index = range(1, len(results) + 1)
        return results_df

    def search(self, query, lang='en'):
        query_terms = self.preprocess(query, lang=lang)
        term_postings, df_dict, num_docs = self.load_block_terms(query_terms)
        query_vector = self.calculate_query_vector(query_terms, df_dict, num_docs)
        return self.cosine_similarity(query_vector, term_postings)


In [21]:
print(data.iloc[64])

track_id                                               02138lFv3Bzncr6ScNbLAF
track_name                                                   Rattle Your Cage
track_artist                                                   Skrizzly Adams
lyrics                      Well I know you've been talking somebody When ...
track_popularity                                                           44
track_album_id                                         3RcttHMKlZ7K1ovxIANPd0
track_album_name                                                    Young Man
track_album_release_date                                           2019-11-15
playlist_name                                       Supernatural Classic Rock
playlist_id                                            6oghIlByD49KFGNmNU8GSH
playlist_genre                                                           rock
playlist_subgenre                                                classic rock
danceability                                                    

In [47]:


block_folder = './blocks/'
search_engine = CosineSimilaritySearch(block_folder)
results_df = search_engine.get_top_k_similar_documents("mayor que yo", data, lang='es', k=5)
print("Top K documentos más similares:")
print(results_df)

['mayor que yo']


KeyError: "None of [Index(['track_name', 'track_artist', 'lyrics', 'Cosine Similarity Score'], dtype='object')] are in the [columns]"